In [ ]:
#@title Dependencies
%%time


In [ ]:
#@title Imports and Definitions
%%time

import os
import sys
from adlfs import AzureBlobFileSystem
import os
from tqdm import notebook
import pandas
import pandas as pd
import praw
import datetime
import asyncpraw
from pmaw import PushshiftAPI

os.environ["AZURE_ACCOUNT_NAME"] = "ajdevreddit"
os.environ["AZURE_TABLE_ENDPOINT"] = "https://ajdevreddit.table.core.windows.net/"
os.environ["AZURE_ACCOUNT_KEY"] = "+9066TCgdeVignRdy50G4qjmNoUJuibl9ERiTGzdV4fwkvgdV3aSVqgLwldgZxj/UpKLkkfXg+3k+AStjFI33Q=="
os.environ["AZURE_STORAGE_CONNECTION_STRING"] = "DefaultEndpointsProtocol=https;AccountName=ajdevreddit;AccountKey=+9066TCgdeVignRdy50G4qjmNoUJuibl9ERiTGzdV4fwkvgdV3aSVqgLwldgZxj/UpKLkkfXg+3k+AStjFI33Q==;BlobEndpoint=https://ajdevreddit.blob.core.windows.net/;QueueEndpoint=https://ajdevreddit.queue.core.windows.net/;TableEndpoint=https://ajdevreddit.table.core.windows.net/;FileEndpoint=https://ajdevreddit.file.core.windows.net/"

class AzureFileStorageAdapter(object):
    def __init__(self):
        self._account_name: str = os.environ["AZURE_ACCOUNT_NAME"]
        self._account_key: str = os.environ["AZURE_ACCOUNT_KEY"]

    def get_file_storage_container(self, container_name: str) -> AzureBlobFileSystem:
        return AzureBlobFileSystem(
            account_name=self._account_name,
            account_key=self._account_key,
            container_name=container_name)

azure_file_store = AzureFileStorageAdapter()

file_system: AzureBlobFileSystem = azure_file_store.get_file_storage_container("data")

In [ ]:
#@title Checking File System
%%time

file_system.ls("data")

In [ ]:
#@title Reddit API Initialization
%%time

reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA', user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)', check_for_async=False)

api: PushshiftAPI = PushshiftAPI(praw=reddit, num_workers=12, batch_size=12, shards_down_behavior="none", jitter="equal")

In [ ]:
#@title Calculate Time Intervals For Training
%%time

start_date_time_stamp: int = int(datetime.datetime.strptime('2023-03-01', '%Y-%m-%d').timestamp())

end_date_time_stamp: int = int(datetime.datetime.today().timestamp())

total_number_days_since_start = (end_date_time_stamp - start_date_time_stamp) // 86400

temp = start_date_time_stamp

intervals = []
while temp < end_date_time_stamp:
    temp = temp + 86400
    intervals.append(temp)

print(f"Total Number Of Intervals: {len(intervals)}\n")

In [ ]:
#@title Collect Data from PRAW
%%time

exists = file_system.ls("data/text/1.AskReddit.parquet")

posts = []

In [ ]:
#@title Filtering Result And Processing To Parquet
%%time

exists = True
if not exists:
    columns = [
    'id',
    'subreddit',
    'title',
    'selftext',
    'author',
    'score',
    'upvote_ratio',
    'ups',
    'subreddit_id',
    'num_comments',
    'permalink']

    foo = df.loc[:, columns]

    foo = foo.where(foo['selftext'] != '[removed]')
    foo = foo.where(foo['selftext'] != '[deleted]')
    foo = foo.where(foo['num_comments'] > 0)
    foo = foo.dropna().reset_index()
    foo = foo.drop('index', axis=1)

    foo = foo.sort_values(by='score', ascending=False).reset_index()
    foo = foo.drop('index', axis=1)

    final_data = []
    out_records = foo.to_dict(orient='records')

    for record in out_records:
        sub = record['subreddit'].__dict__['display_name']
        record['subreddit'] = sub
        author = record['author'].__dict__['name']
        record['author'] = author
        final_data.append(record)

    bar = pandas.DataFrame(data=final_data)
    bar.to_parquet("data/text/1.AskReddit.parquet", engine='pyarrow', filesystem=file_system)

In [ ]:
submission_data: pandas.DataFrame = pd.read_parquet("data/text/1.AskReddit.parquet", engine='pyarrow', filesystem=file_system)
display(submission_data)

In [ ]:
#@title Obtain Comments For Each Sub
%%time

submission_id_list = list(submission_data['id'])

for submission_id in notebook.tqdm(submission_id_list, desc="Getting Submission"):
    out_path = f"/*{submission_id}.parquet"
    if file_system.exists(out_path):
        continue
    submission = reddit.submission(submission_id)
    submission.comment_sort = 'top'

    comments_for_submission = []
    top_level_comments = submission.comments.list()

    for comment in top_level_comments:
        try:
            comment.author.__dict__['name']
        except AttributeError:
            continue
        data = {
             'id': comment.id,
             'parent_id': comment.parent_id,
             'link_id': comment.link_id,
             'submission_id': submission_id,
             'body': comment.body,
             'author': comment.author.__dict__['name'],
             'score': comment.score
            }
        comments_for_submission.append(data)
    temp_df = pandas.DataFrame(data=comments_for_submission)
    temp_df.to_parquet(out_path, engine='pyarrow', filesystem=file_system)